In [1]:
!pip install transformers langchain faiss-gpu sentence_transformers --quiet

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "cerebras/Cerebras-GPT-13B"
cache_dir = "/home/ec2-user/SageMaker/tmp"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import pipeline


pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer,
    max_new_tokens=100, early_stopping=True, no_repeat_ngram_size=2
)

In [4]:
from langchain.llms import HuggingFacePipeline


llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate

# HypotheticalDocumentEmbedderの準備
embeddings = HypotheticalDocumentEmbedder.from_llm(
    llm=llm, 
    base_embeddings=HuggingFaceEmbeddings(), 
    prompt_key="web_search"
)

In [9]:
web_search_template = """Please write a passage to answer the question 
Question: {QUESTION}
Passage:"""

In [10]:
# テキストから埋め込みを生成
result = embeddings.embed_query("Where is the Taj Mahal?")
print(len(result))
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


768
[0.08058663457632065, -0.06910695135593414, -0.016902122646570206, 0.03171413391828537, -0.05484837666153908, 0.015202519483864307, 0.017748141661286354, 0.030727369710803032, -0.02518908493220806, -0.005670362152159214, 0.057341400533914566, 0.015774888917803764, 0.09196390211582184, -0.042266737669706345, 0.027663150802254677, -0.03561811521649361, 0.0039898366667330265, -0.02574714459478855, -0.01699313521385193, 0.022647878155112267, -0.0299875158816576, 0.06678295135498047, -0.028813261538743973, -0.008927821181714535, -0.00574614480137825, 0.003108639270067215, -0.01733638532459736, -0.01968350075185299, -0.004608316347002983, -0.021546486765146255, 0.007901503704488277, 0.021001987159252167, -0.032862383872270584, -0.048638612031936646, 1.9273816178611014e-06, -0.006732514593750238, 0.02057644911110401, -0.013797885738313198, 0.016286391764879227, -0.07957051694393158, -0.05189121514558792, 0.08955163508653641, 0.016307003796100616, -0.007625294383615255, 0.00113440642599016

In [13]:
# 質問応答
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

# ドキュメントの読み込み
with open("bocchi.txt") as f:
    bocchi_txt = f.read()
text_splitter = CharacterTextSplitter(
    chunk_size=50, 
    chunk_overlap=0, 
    separator="。"
)
texts = text_splitter.split_text(bocchi_txt)
print(len(texts))
print(texts)

Created a chunk of size 86, which is longer than the specified 50
Created a chunk of size 74, which is longer than the specified 50
Created a chunk of size 69, which is longer than the specified 50
Created a chunk of size 53, which is longer than the specified 50
Created a chunk of size 59, which is longer than the specified 50
Created a chunk of size 69, which is longer than the specified 50


14
['後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める', 'ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業', '結局、友達を作るという当初の目標は達成することはできずじまいだった', 'そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた', 'ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし', '黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる', 'ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた', '結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった', 'そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る', '紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる', 'ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう', '絶望するひとりは、そこで行き倒れた廣井きくりと遭遇', '彼女の計らいで突発的な路上ライブを行なって、観客を魅了する', 'きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。']


In [14]:
query = "ギタリストは誰?"

# 類似文章の検索
docsearch = FAISS.from_texts(texts, embeddings)
docs = docsearch.similarity_search(query, k=1)
print(docs[0].page_content)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


結局、友達を作るという当初の目標は達成することはできずじまいだった


In [16]:
# 質問応答の実行
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'output_text': ' 組織の中で、どのような人が知りたいのかを確認すべきだと思う\n  私は、秘密の会議を開催すれば、この質問を解決すらでもです。\nContext: The secret meeting is to'}

In [ ]:
# 結果は全然だめ